In [6]:
import tensorflow as tf

## 0.前言

* 对于需要创建自定义逻辑的网络层，需要继承自`layers.Layer`基类
* 创建自定义网络类时，需要继承自`keras.Model`类

## 1.自定义网络层

对于自定义的网络层，至少需要实现初始化`__init__`和前向传播逻辑`call`方法。

例子：<br>
实现一个没有偏置向量的全连接层(bias为0)，同时固定激活函数为ReLU函数：


In [7]:
from tensorflow import keras
class MyDense(keras.layers.Layer):
    ''' 自定义网络层
    '''
    def __init__(self,inp_dim,outp_dim):
        super(MyDense, self).__init__()
        # 创建权值张量并添加到类管理列表中，设置为需要优化
        # self.kernel=self.add_variable('w',[inp_dim,outp_dim],trainable=True)
        self.kernel=self.add_weight('w',[inp_dim,outp_dim],trainable=True)

    def call(self, inputs, training=None):
        # X@W
        out=inputs@self.kernel
        # 执行激活函数
        out=tf.nn.relu(out)
        return out


* `self.add_variable(name,shape)`将权值张量添加到类管理列表后会返回张量W的Python引用，而变量名name由TensorFlow内部维护，使用的比较少。
* `call(inputs,training=None)`中inputs代表输入，training参数指定模型状态，True为训练模式，None或False为测试模式


In [8]:
net=MyDense(4,3)
print(net.variables)
print(net.trainable_variables)

[<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.37874138,  0.5557107 ,  0.53060985],
       [ 0.29093695, -0.1935488 , -0.5136733 ],
       [-0.47883105,  0.3693993 , -0.18341893],
       [-0.43808645, -0.13748914, -0.03073812]], dtype=float32)>]
[<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.37874138,  0.5557107 ,  0.53060985],
       [ 0.29093695, -0.1935488 , -0.5136733 ],
       [-0.47883105,  0.3693993 , -0.18341893],
       [-0.43808645, -0.13748914, -0.03073812]], dtype=float32)>]


## 2.自定义网络

对于自定义网络，也需要实现初始化`__init__`和前向传播逻辑`call`方法。

自定义网络类可以和其他标准类一样，可被Sequential容器封装。我们先通过自定义网络层来堆叠一个网络：

In [9]:
network=keras.Sequential([
    MyDense(28*28,256),
    MyDense(256,128),
    MyDense(128,64),
    MyDense(64,32),
    MyDense(32,10),
])
network.build(input_shape=(None,28*28))
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_12 (MyDense)        (None, 256)               200704    
_________________________________________________________________
my_dense_13 (MyDense)        (None, 128)               32768     
_________________________________________________________________
my_dense_14 (MyDense)        (None, 64)                8192      
_________________________________________________________________
my_dense_15 (MyDense)        (None, 32)                2048      
_________________________________________________________________
my_dense_16 (MyDense)        (None, 10)                320       
Total params: 244,032
Trainable params: 244,032
Non-trainable params: 0
_________________________________________________________________


下面创建一个自定义网络类实现上述等价的效果：

In [10]:
class MyModel(keras.Model):
    # 自定义网络类，继承Model基类
    def __init__(self):
        super(MyModel, self).__init__()
        # 完成网络内需要的网络层的创建工作
        self.fc1=MyDense(28*28,256)
        self.fc2=MyDense(256,128)
        self.fc3=MyDense(128,64)
        self.fc4=MyDense(64,32)
        self.fc5=MyDense(32,10)

    def call(self, inputs, training=None, mask=None):
        # 自定义前向运算逻辑
        x=self.fc1(inputs)
        x=self.fc2(x)
        x=self.fc3(x)
        x=self.fc4(x)
        x=self.fc5(x)
        return x

model=MyModel()
model.build(input_shape=(None,28*28))
model.summary()

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_17 (MyDense)        multiple                  200704    
_________________________________________________________________
my_dense_18 (MyDense)        multiple                  32768     
_________________________________________________________________
my_dense_19 (MyDense)        multiple                  8192      
_________________________________________________________________
my_dense_20 (MyDense)        multiple                  2048      
_________________________________________________________________
my_dense_21 (MyDense)        multiple                  320       
Total params: 244,032
Trainable params: 244,032
Non-trainable params: 0
_________________________________________________________________


虽然使用`Sequential`实现同样的效果，但自定义网络的前向设计更加自由，更通用。

In [ ]:
import os
pid=os.getpid()
!kill -9 $pid
